In [17]:
import requests
from bs4 import BeautifulSoup

from datetime import datetime

In [38]:
LAST_SET_RELEASE = datetime(year=2025, month=8, day=1)

---

In [19]:
### Tournament Example

URL = f'https://www.mtgtop8.com/event?e=74153&f=PAU'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

info_div = soup.find("div", style="margin-bottom:5px;")
info_div

<div style="margin-bottom:5px;">25 players - 22/09/25</div>

---

In [20]:
def scrape_decklist(deck_id: str, metagame: dict, skip_online=False) -> dict:
    
    if not metagame:
        metagame = {'cards': {}, 'decks': {}}
    
    URL = f'https://www.mtgtop8.com/event{deck_id}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    event_title_parsed = [item.get_text(strip=True) for item in soup.find_all("div", class_="event_title")]
    
    if '←' in event_title_parsed:
        pos = 2
    else:
        pos = 1
        
    deck_player = soup.find_all("div", class_="event_title")[pos].get_text(strip=True)
    deck_name = ' '.join(deck_player.split(' -')[0].split(' ')[1:])
    player_name = deck_player.split(' -')[1]
    
    if len(player_name.split(' ')) == 1 and skip_online:
        return metagame
    
    if deck_name not in metagame['decks']:
        metagame['decks'][deck_name] = 0
    metagame['decks'][deck_name] += 1

    deck_list = {}
    for div in soup.find_all('div', class_='deck_line hover_tr'):
        quantity, card_name = int(div.text.split(' ')[0]), ' '.join(div.text.split(' ')[1:])

        if card_name not in deck_list:
            deck_list[card_name] = quantity
        else:
            deck_list[card_name] += quantity

        if card_name not in metagame['cards']:
            metagame['cards'][card_name] = {}

        if deck_name not in metagame['cards'][card_name]:
            metagame['cards'][card_name][deck_name] = deck_list[card_name]

        elif metagame['cards'][card_name][deck_name] < deck_list[card_name]:
            metagame['cards'][card_name][deck_name] = deck_list[card_name]

    return metagame

In [21]:
metagame = scrape_decklist('?e=73865&d=760353&f=PAU', {}, skip_online=True)
metagame

{'cards': {}, 'decks': {}}

In [22]:
metagame = scrape_decklist('?e=73865&d=760354&f=PAU', metagame)
metagame

{'cards': {'Island ': {'Mono Blue Aggro': 18},
  'Brinebarrow Intruder ': {'Mono Blue Aggro': 4},
  'Faerie Miscreant ': {'Mono Blue Aggro': 4},
  'Faerie Seer ': {'Mono Blue Aggro': 4},
  'Harrier Strix ': {'Mono Blue Aggro': 2},
  'Illvoi Galeblade ': {'Mono Blue Aggro': 2},
  'Moon-Circuit Hacker ': {'Mono Blue Aggro': 4},
  'Ninja of the Deep Hours ': {'Mono Blue Aggro': 4},
  'Spellstutter Sprite ': {'Mono Blue Aggro': 4},
  'Counterspell ': {'Mono Blue Aggro': 4},
  'Dispel ': {'Mono Blue Aggro': 3},
  'Of One Mind ': {'Mono Blue Aggro': 4},
  'Snap ': {'Mono Blue Aggro': 3},
  'Bind the Monster ': {'Mono Blue Aggro': 3},
  'Annul ': {'Mono Blue Aggro': 4},
  'Azure Fleet Admiral ': {'Mono Blue Aggro': 1},
  'Blue Elemental Blast ': {'Mono Blue Aggro': 4},
  'Hydroblast ': {'Mono Blue Aggro': 2},
  'Steel Sabotage ': {'Mono Blue Aggro': 1}},
 'decks': {'Mono Blue Aggro': 1}}

In [23]:
metagame = scrape_decklist('?e=73865&d=760363&f=PAU', metagame)
metagame

{'cards': {'Island ': {'Mono Blue Aggro': 18, 'Mono Blue Delver': 16},
  'Brinebarrow Intruder ': {'Mono Blue Aggro': 4},
  'Faerie Miscreant ': {'Mono Blue Aggro': 4},
  'Faerie Seer ': {'Mono Blue Aggro': 4},
  'Harrier Strix ': {'Mono Blue Aggro': 2},
  'Illvoi Galeblade ': {'Mono Blue Aggro': 2},
  'Moon-Circuit Hacker ': {'Mono Blue Aggro': 4},
  'Ninja of the Deep Hours ': {'Mono Blue Aggro': 4},
  'Spellstutter Sprite ': {'Mono Blue Aggro': 4},
  'Counterspell ': {'Mono Blue Aggro': 4, 'Mono Blue Delver': 4},
  'Dispel ': {'Mono Blue Aggro': 3, 'Mono Blue Delver': 3},
  'Of One Mind ': {'Mono Blue Aggro': 4},
  'Snap ': {'Mono Blue Aggro': 3},
  'Bind the Monster ': {'Mono Blue Aggro': 3},
  'Annul ': {'Mono Blue Aggro': 4, 'Mono Blue Delver': 4},
  'Azure Fleet Admiral ': {'Mono Blue Aggro': 1},
  'Blue Elemental Blast ': {'Mono Blue Aggro': 4, 'Mono Blue Delver': 1},
  'Hydroblast ': {'Mono Blue Aggro': 2, 'Mono Blue Delver': 4},
  'Steel Sabotage ': {'Mono Blue Aggro': 1, 'Mo

In [24]:
def scrape_tournament(tournament_id: str, metagame: dict, skip_online=False, date_from=None) -> dict:
    URL = f'https://www.mtgtop8.com/{tournament_id}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    if len(soup.find("div", style="margin-bottom:5px;").get_text(strip=True).split(' - ')) != 2:
        return metagame
    
    players, date = soup.find("div", style="margin-bottom:5px;").get_text(strip=True).split(' - ')
    date = datetime.strptime(date, '%d/%m/%y')
    players = int(players.split(' ')[0])
    
    if date_from and date < date_from:
        print(f'Event skipped because happened in the past: event date={date}')
        return metagame
    
    visited_decks = {}
    for a in soup.find_all('a', href=True):
        if a['href'].endswith('PAU') and a['href'].startswith('?'):
            print('Decklist url:', a['href'])
            if a['href'] not in visited_decks:
                metagame = scrape_decklist(a['href'], metagame, skip_online)
                visited_decks[a['href']] = True
    return metagame

In [25]:
metagame = scrape_tournament('event?e=74153&f=PAU', {}, datetime(year=2025, month=8, day=1))
metagame

Decklist url: ?e=74153&d=762614&f=PAU
Decklist url: ?e=74153&d=762613&f=PAU
Decklist url: ?e=74153&d=762613&f=PAU
Decklist url: ?e=74153&d=762614&f=PAU
Decklist url: ?e=74153&d=762614&f=PAU
Decklist url: ?e=74153&d=762615&f=PAU
Decklist url: ?e=74153&d=762615&f=PAU
Decklist url: ?e=74153&d=762616&f=PAU
Decklist url: ?e=74153&d=762616&f=PAU
Decklist url: ?e=74153&d=762617&f=PAU
Decklist url: ?e=74153&d=762617&f=PAU
Decklist url: ?e=74153&d=762619&f=PAU
Decklist url: ?e=74153&d=762619&f=PAU
Decklist url: ?e=74153&d=762620&f=PAU
Decklist url: ?e=74153&d=762620&f=PAU
Decklist url: ?e=74153&d=762618&f=PAU
Decklist url: ?e=74153&d=762618&f=PAU


{'cards': {'Great Furnace ': {'Red Deck Wins': 4},
  'Mountain ': {'Red Deck Wins': 14, 'Jund Wildfire': 2},
  'Burning-Tree Emissary ': {'Red Deck Wins': 4},
  'Clockwork Percussionist ': {'Red Deck Wins': 4},
  'Gingerbrute ': {'Red Deck Wins': 3},
  'Goblin Bushwhacker ': {'Red Deck Wins': 4},
  'Goblin Tomb Raider ': {'Red Deck Wins': 4},
  'Voldaren Epicure ': {'Red Deck Wins': 4},
  'Chain Lightning ': {'Red Deck Wins': 3},
  'Galvanic Blast ': {'Red Deck Wins': 4, 'Jund Wildfire': 4},
  'Lightning Bolt ': {'Red Deck Wins': 4},
  'Rally at the Hornburg ': {'Red Deck Wins': 4},
  'Reckless Impulse ': {'Red Deck Wins': 2},
  "Wrenn's Resolve ": {'Red Deck Wins': 2},
  'Cast into the Fire ': {'Red Deck Wins': 4},
  'End the Festivities ': {'Red Deck Wins': 4},
  'Pyroblast ': {'Red Deck Wins': 4, 'Jund Wildfire': 2},
  'Relic of Progenitus ': {'Red Deck Wins': 3},
  'Contaminated Aquifer ': {'Dimir Control': 2},
  'Island ': {'Dimir Control': 5},
  'Mistvault Bridge ': {'Dimir Contr

In [26]:
def scrape_website(URL: str, metagame: dict, skip_online=False, date_from=None) -> dict:
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        if a['href'].startswith('event'):
            print(a['href'])
            metagame = scrape_tournament(a['href'], metagame, skip_online, date_from)
    return metagame

In [36]:
metagame_all = scrape_website('https://www.mtgtop8.com/format?f=PAU', {}, skip_online=False, date_from=LAST_SET_RELEASE)
metagame_all

event?e=74112&f=PAU
Decklist url: ?e=74112&d=762292&f=PAU
Decklist url: ?e=74112&d=762291&f=PAU
Decklist url: ?e=74112&d=762291&f=PAU
Decklist url: ?e=74112&d=762292&f=PAU
Decklist url: ?e=74112&d=762292&f=PAU
Decklist url: ?e=74112&d=762294&f=PAU
Decklist url: ?e=74112&d=762294&f=PAU
Decklist url: ?e=74112&d=762293&f=PAU
Decklist url: ?e=74112&d=762293&f=PAU
Decklist url: ?e=74112&d=762298&f=PAU
Decklist url: ?e=74112&d=762298&f=PAU
Decklist url: ?e=74112&d=762295&f=PAU
Decklist url: ?e=74112&d=762295&f=PAU
Decklist url: ?e=74112&d=762297&f=PAU
Decklist url: ?e=74112&d=762297&f=PAU
Decklist url: ?e=74112&d=762296&f=PAU
Decklist url: ?e=74112&d=762296&f=PAU
Decklist url: ?e=74112&d=762299&f=PAU
Decklist url: ?e=74112&d=762299&f=PAU
Decklist url: ?e=74112&d=762302&f=PAU
Decklist url: ?e=74112&d=762302&f=PAU
Decklist url: ?e=74112&d=762303&f=PAU
Decklist url: ?e=74112&d=762303&f=PAU
Decklist url: ?e=74112&d=762301&f=PAU
Decklist url: ?e=74112&d=762301&f=PAU
Decklist url: ?e=74112&d=76230

{'cards': {'Great Furnace ': {'Burn': 4,
   'Boros Aggro': 4,
   'Affinity': 2,
   'Grixis Affinity': 3,
   'Red Deck Wins': 4,
   'Rakdos Madness Burn': 4,
   'Mono R Pingers': 4},
  'Mountain ': {'Burn': 20,
   'Boros Aggro': 3,
   'Gruul Aggro': 2,
   'Jund': 2,
   'Mono Red Pingers': 18,
   'Madness Burn': 18,
   'Jund Wildfire': 2,
   'Red Deck Wins': 17,
   'Mono Green Aggro': 2,
   'Gates': 2,
   'Boros Synthesizer': 5,
   'Gruul Midrange': 2,
   'Rakdos Madness Burn': 7,
   'Rakdos Cim': 7,
   'Tarmoless Jund': 1,
   'Mono R Pingers': 14},
  'Clockwork Percussionist ': {'Burn': 4,
   'Red Deck Wins': 4,
   'Mono R Pingers': 4},
  'Goblin Tomb Raider ': {'Burn': 4,
   'Red Deck Wins': 4,
   'Rakdos Madness Burn': 4,
   'Mono R Pingers': 4},
  'Kessig Flamebreather ': {'Burn': 4,
   'Mono Red Pingers': 4,
   'Madness Burn': 4,
   'Red Deck Wins': 4,
   'Mono R Pingers': 4},
  'Voldaren Epicure ': {'Burn': 4,
   'Madness Burn': 4,
   'Red Deck Wins': 4,
   'Rakdos Madness Burn': 4

In [28]:
metagame_real = scrape_website('https://www.mtgtop8.com/format?f=PAU', {}, skip_online=True, date_from=LAST_SET_RELEASE)
metagame_real

event?e=74112&f=PAU
Decklist url: ?e=74112&d=762292&f=PAU
Decklist url: ?e=74112&d=762291&f=PAU
Decklist url: ?e=74112&d=762291&f=PAU
Decklist url: ?e=74112&d=762292&f=PAU
Decklist url: ?e=74112&d=762292&f=PAU
Decklist url: ?e=74112&d=762294&f=PAU
Decklist url: ?e=74112&d=762294&f=PAU
Decklist url: ?e=74112&d=762293&f=PAU
Decklist url: ?e=74112&d=762293&f=PAU
Decklist url: ?e=74112&d=762298&f=PAU
Decklist url: ?e=74112&d=762298&f=PAU
Decklist url: ?e=74112&d=762295&f=PAU
Decklist url: ?e=74112&d=762295&f=PAU
Decklist url: ?e=74112&d=762297&f=PAU
Decklist url: ?e=74112&d=762297&f=PAU
Decklist url: ?e=74112&d=762296&f=PAU
Decklist url: ?e=74112&d=762296&f=PAU
Decklist url: ?e=74112&d=762299&f=PAU
Decklist url: ?e=74112&d=762299&f=PAU
Decklist url: ?e=74112&d=762302&f=PAU
Decklist url: ?e=74112&d=762302&f=PAU
Decklist url: ?e=74112&d=762303&f=PAU
Decklist url: ?e=74112&d=762303&f=PAU
Decklist url: ?e=74112&d=762301&f=PAU
Decklist url: ?e=74112&d=762301&f=PAU
Decklist url: ?e=74112&d=76230

{'cards': {'Great Furnace ': {'Burn': 4,
   'Boros Aggro': 4,
   'Affinity': 2,
   'Red Deck Wins': 4,
   'Rakdos Madness': 4,
   'Mono Red Synthesizer': 4,
   'Grixis Affinity': 3,
   'Rakdos Madness Burn': 4,
   'Mono R Pingers': 4},
  'Mountain ': {'Burn': 20,
   'Boros Aggro': 3,
   'Gruul Aggro': 2,
   'Jund': 2,
   'Red Deck Wins': 14,
   'Jund Wildfire': 2,
   'Rakdos Madness': 2,
   'Gruul Monsters': 2,
   'Mono Red Madness': 18,
   'Mono Red Synthesizer': 13,
   'Mono Red Pingers': 18,
   'Madness Burn': 18,
   'Mono Green Aggro': 2,
   'Gruul Midrange': 2,
   'Rakdos Madness Burn': 2,
   'Rakdos Cim': 7,
   'Tarmoless Jund': 1,
   'Mono R Pingers': 14},
  'Clockwork Percussionist ': {'Burn': 4,
   'Red Deck Wins': 4,
   'Mono Red Madness': 4,
   'Mono Red Synthesizer': 4,
   'Mono R Pingers': 4},
  'Goblin Tomb Raider ': {'Burn': 4,
   'Red Deck Wins': 4,
   'Mono Red Synthesizer': 4,
   'Mono R Pingers': 4},
  'Kessig Flamebreather ': {'Burn': 4,
   'Red Deck Wins': 4,
   'M

In [35]:
metagame_all['decks']

{'Burn': 15,
 'Gates': 10,
 'Boros Aggro': 7,
 'Mono Black Aggro': 12,
 'Affinity': 9,
 'Golgari Garden': 4,
 'Balustrade Spy': 21,
 'Esper Affinity': 4,
 'Gruul Aggro': 10,
 'Jund': 14,
 'Mono Blue Aggro': 14,
 'Gruul Midrange': 5,
 'Rakdos Madness Burn': 5,
 'Madness Burn': 9,
 'Mono Blue Delver': 19,
 'Orzhov Aggro': 4,
 'Red Deck Wins': 21,
 'Dimir Terror': 1,
 'Golgari / Jund Garden': 10,
 'Mono Black Burn': 1,
 'Mono Green Aggro': 6,
 'High Tide': 8,
 'Golgari Gardens': 2,
 'Cryo Gates': 2,
 'Elves': 10,
 'Jund Wildfire': 16,
 'Izzet Terror': 4,
 'Rakdos Madness': 2,
 'Gruul Monsters': 2,
 'Mono Blue Faeries': 2,
 'Mono Red Madness': 2,
 'Mono Red Synthesizer': 2,
 'Walls Combo': 5,
 'Mono Blue Terror': 2,
 'Walls Spy': 2,
 'Mono Red Pingers': 2,
 'Dimir Delver': 2,
 'Grixis Affinity': 13,
 'Ephemerate': 2,
 'Dimir Control': 3,
 'UR Ninja': 2,
 'Boros Synthesizer': 1,
 'Caw Gates': 1,
 'Urzatron': 4,
 'Rakdos Cim': 2,
 'Una Grande Scoteca': 2,
 'Monou Vigliacche': 2,
 'Tutte Le B

In [31]:
metagame_real['decks']

{'Burn': 14,
 'Gates': 8,
 'Boros Aggro': 6,
 'Mono Black Aggro': 6,
 'Affinity': 8,
 'Golgari Garden': 4,
 'Balustrade Spy': 20,
 'Esper Affinity': 4,
 'Gruul Aggro': 10,
 'Jund': 14,
 'Mono Blue Delver': 10,
 'High Tide': 6,
 'Mono Blue Aggro': 8,
 'Red Deck Wins': 12,
 'Golgari Gardens': 2,
 'Cryo Gates': 2,
 'Elves': 8,
 'Jund Wildfire': 12,
 'Izzet Terror': 4,
 'Rakdos Madness': 2,
 'Gruul Monsters': 2,
 'Mono Blue Faeries': 2,
 'Mono Red Madness': 2,
 'Mono Red Synthesizer': 2,
 'Walls Combo': 4,
 'Mono Blue Terror': 2,
 'Walls Spy': 2,
 'Mono Red Pingers': 2,
 'Dimir Delver': 2,
 'Madness Burn': 4,
 'Grixis Affinity': 12,
 'Dimir Control': 2,
 'Golgari / Jund Garden': 2,
 'Mono Green Aggro': 4,
 'UR Ninja': 2,
 'Orzhov Aggro': 2,
 'Gruul Midrange': 2,
 'Rakdos Madness Burn': 2,
 'Urzatron': 2,
 'Rakdos Cim': 2,
 'Una Grande Scoteca': 2,
 'Monou Vigliacche': 2,
 'Tutte Le Bisce Del Presidente': 2,
 'Acqua Frissante Tutti I Giorni': 2,
 'Esperienze': 2,
 'Gruul': 2,
 'Tarmoless Ju